In [1]:
import requests
from IPython.core.display import display, HTML
from bs4 import BeautifulSoup
from personal import *

url = "http://127.0.0.1:5000"
# url = "https://turnitin-api.herokuapp.com"


s = requests.Session()

In [3]:
login_result = s.post(url + "/login", json={
    "email": USERNAME,
    "password": PASSWORD
})
auth = login_result.json()

classes_result = s.post(url + "/classes", json=auth)
classes = classes_result.json()
for c in classes:
    if "World Lit" in c["title"]:
        world_lit = c
print(world_lit)

first_class_data = dict(auth, url=world_lit["url"])
assignments_result = s.post(url + "/assignments", json=first_class_data)
assignments = assignments_result.json()
assignments[25]

{'title': 'World Literature: Period 5', 'url': 'https://www.turnitin.com/s_class_portfolio.asp?r=96.4763860571541&svr=37&lang=en_us&aid=5245&cid=22703844'}


{'aid': '93021908',
 'dates': {'due': '06/30/2020 23:59:00',
  'post': '06/12/2020 00:00:00',
  'start': '06/04/2020 15:35:00'},
 'file': 'void',
 'info': 'No additional instructions have been entered for this assignment. Late paper submissions are allowed for this assignment. Resubmissions are not allowed for this assignment. Similarity reports are available immediately after submission.',
 'oid': 'void',
 'submission': 't_submit.asp?r=39.8521947242529&svr=51&lang=en_us&aid=93021908',
 'title': "Revision 2: Kento's Test Assignment (for Kento only)"}

In [16]:
len(assignments)

29

In [5]:
ass = assignments[25]
aid = ass["aid"]
cookies = auth["auth"]
author_first = "Kento"
author_last = "Nishi"
submission_title = "Custom turnitin API"

In [7]:
referer_url = "https://www.turnitin.com/" + ass["submission"]
submit_url = "https://www.turnitin.com/t_submit.asp"
query = {"aid": aid, "session-id": cookies["session-id"], "lang": "en_us"}
form_data = dict(
    async_request=1,
    author_first=author_first, 
    author_last=author_last,
    title=submission_title,
    filename="SampleSubmission.docx",
    referer=referer_url,
    # userfile='',
    db_doc='',
    dropbox_filename='',
    google_doc='',
    google_auth_uri='',
    token='',
    submit_button=''
)

fi = {"userfile": ("Document.docx",open("Document.docx", 'rb'), "application/vnd.openxmlformats-officedocument.wordprocessingml.document"),
"Content-Disposition": "form-data; name=\"file\"; filename=\"SampleSubmission.docx\"",
"Content-Type": "application/vnd.openxmlformats-officedocument.wordprocessingml.document"}


In [8]:
[{i: form_data[i]} for i in form_data if i != "userfile"]

[{'async_request': 1},
 {'author_first': 'Kento'},
 {'author_last': 'Nishi'},
 {'title': 'Custom turnitin API'},
 {'filename': 'SampleSubmission.docx'},
 {'referer': 'https://www.turnitin.com/t_submit.asp?r=39.8521947242529&svr=51&lang=en_us&aid=93021908'},
 {'db_doc': ''},
 {'dropbox_filename': ''},
 {'google_doc': ''},
 {'google_auth_uri': ''},
 {'token': ''},
 {'submit_button': ''}]

In [9]:
r = s.post(submit_url, data=form_data, files=fi, headers={"accept": "application/json"}, params=query, cookies=cookies)

In [10]:
r.headers["content-type"]

'application/json'

In [7]:
r.json()

{'error_codes': None,
 'errors': None,
 'uuid': 'F1D7FA8E-B98E-11EA-B246-ACF57A601489'}

In [32]:
rr = s.post("https://www.turnitin.com/panda/get_submission_metadata.asp", params={
    "session-id": cookies["session-id"],
    "lang": "en_us",
    "skip_ready_check": 0,
    "uuid": r.json()["uuid"]
})
rr.json()

{'word_count': 180,
 'status': 1,
 'page_count': 5,
 'image_url_stub': '/panda/get_page_image.asp?m=6ddcec3cd528448aa1e655c43a31e9a9&session-id=0c6322808aac485dad23bd9083604d6c&submission_id=oid:1:1794083027&page=',
 'char_count': 783,
 'file_size': '10.46M',
 'uuid': '5653DE6E-B990-11EA-879B-FA479F8646E8',
 'download_url': None,
 'file_name': 'SampleSubmission.docx'}

In [39]:
confirmation_url = "https://www.turnitin.com/submit_confirm.asp"
c_query = {
    "lang": "en_us",
    "session-id": cookies["session-id"],
    "data-state": "confirm",
    "uuid": rr.json()["uuid"]
}
rrr = s.post(confirmation_url, headers={
    "accept": "application/json",
    "origin": "https://www.turnitin.com",
    "referer": referer_url    
}, params=query, cookies=cookies)
rrr.content, rrr.headers["content-type"]

(b'null', 'text/html; charset=UTF-8')

In [41]:
rrr = s.post(f"{confirmation_url}?lang=en_us&sessionid={c_query['session-id']}&data-state=confirm&uuid={c_query['uuid']}")

In [43]:
rrr.text

'{"oid":"1350956643","submission_date":"2020年06月28日 03:46PM (UTC-0700)"}'

In [38]:
c_query

{'lang': 'en_us',
 'session-id': '0c6322808aac485dad23bd9083604d6c',
 'data-state': 'confirm',
 'uuid': '5653DE6E-B990-11EA-879B-FA479F8646E8'}